# Step 1: Import CPTAC Data and Define Correlation Functions

In [1]:
import cptac
import cptac.utils as ut
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statistics as st
import load_data
import get_correlations

In [2]:
#Make docstrings for what is required
def get_single_gene_correlations_orig(gene, cancer_type_list,type_dict,input_tissue_type = "tumor"):
    correlation_list = []
    type_list = []
    cancer_index = 0
    for cancer in cancer_type_list:
        correlation,pval,num_samples = get_correlations.correlation_list(cancer, [gene],tissue_type = input_tissue_type)[0]
        if len(correlation) == 0:
            continue
        else:
            correlation_list.append(correlation[0])
            type_list.append(type_dict[cancer])
        cancer_index += 1
    return correlation_list, type_list, num_samples

In [3]:
def make_df_with_genes(gene_list,cancer_list,name_dict,tissue_type):
    all_genes = []
    all_correlations = []
    all_cancer_types = []
    for gene in gene_list:
        gene_corr_list,type_list, num_samples = get_single_gene_correlations(gene,cancer_list,name_dict,tissue_type)
        gene_x,gene_y = get_x_y(gene_corr_list)
        all_genes.extend(gene_x)
        all_correlations.extend(gene_y)
        all_cancer_types.extend(type_list)
    df = pd.DataFrame(list(zip(all_genes,all_correlations,all_cancer_types)), columns =['Gene', 'Correlation','Cancer Type'])
    return(df)

In [4]:
def get_x_y(correlation_list):
    x = []
    y = []
    for item in correlation_list:
        x.append(item[0])
        y.append(item[1])
    return x,y

# Step 2: Define Desired Cancer Types

In [5]:
brca = cptac.Brca()
ccrcc = cptac.Ccrcc()
colon = cptac.Colon()
en = cptac.Endometrial()
gbm = cptac.Gbm()
luad = cptac.Luad()
ovarian = cptac.Ovarian()
hnscc  = cptac.Hnscc()
lscc = cptac.Lscc()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (<ipython-input-5-0467c23f8dc0>, line 5)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-5-0467c23f8dc0>, line 9)


In [6]:
cancer_list = [brca,ccrcc,colon,en,gbm,luad,ovarian,hnscc,lscc]

In [7]:
type_dict = {brca:"brca",ccrcc:"ccrcc",colon:"colon",en:"endometrial",gbm:"gbm",luad:"luad",
                  ovarian:"ovarian",hnscc:"hnscc",lscc:"lscc"}

# Step 3: Pick Genes and Make Dataframe

The genes shown here were picked from Bailey et al. found [here](https://pubmed.ncbi.nlm.nih.gov/29625053/)

In [8]:
gene_list = ['ABL1','BRAF','FAT1','IDH1','TP53','BRCA2','KMT2D','KRAS','PIK3CA','PTEN','SETD2','SPTA1']

In [9]:
def get_df_with_type(df,input_type):
    return(df.loc[df['Type']==input_type])

In [10]:
def find_mut_tumor(cancer_type, gene):
    gene_multi = cancer_type.multi_join({'proteomics': gene, 'transcriptomics': gene, 'somatic_mutation': gene}, tissue_type = 'tumor')
    normal = cancer_type.multi_join({'proteomics': gene, 'transcriptomics': gene}, tissue_type= 'normal', flatten = True)
    normal.columns = ['proteomics', 'transcriptomics']
    trans = list(gene_multi[str(gene +"_transcriptomics")])
    prot = list(gene_multi[str(gene + '_proteomics')])
    group = []
    for i in gene_multi[str(gene + '_Mutation_Status')]:
        if type(i) == str:
            group.append("mutation")
        else:
            group.append("wt") 
    group.extend(['normal']*len(normal))
    prot.extend(list(normal['proteomics']))
    trans.extend(list(normal['transcriptomics']))
    gene_df = pd.DataFrame({'Type': group, 'Proteomics': prot, 'Transcriptomics': trans})
    gene_df = gene_df.dropna()
    return gene_df

In [11]:
#Make docstrings for what is required
def get_single_gene_correlations(gene, cancer_type_list,type_dict,input_tissue_type = "tumor"):
    correlation_list = []
    type_list = []
    cancer_index = 0
    for cancer in cancer_type_list:
        corr_df = find_mut_tumor(cancer,gene)
        if input_tissue_type == "tumor":
            corr_df = get_df_with_type(corr_df,"wt")
        else:
            corr_df = get_df_with_type(corr_df,"normal")
        correlation = corr_df.corr(method="pearson").loc["Proteomics"][1]
        if len(correlation) == 0:
            continue
        else:
            correlation_list.append(correlation[0])
            type_list.append(type_dict[cancer])
        cancer_index += 1
    return correlation_list, type_list

In [12]:
my_df = make_df_with_genes(gene_list,cancer_list,type_dict,tissue_type = "tumor")

ValueError: arrays must all be same length

In [ ]:
my_df

# Step 4: Plot Dataframes

In [ ]:
hue_order = ['ccrcc','hnscc','lscc','luad','brca','endometrial','gbm','colon','ovarian']

In [ ]:
plt = sns.stripplot(x="Correlation",y="Gene",hue="Cancer Type",hue_order=hue_order,data=my_df,jitter=0)
plt.legend_.remove()
plt.set(xlim=(-.5,1))
plt

In [ ]:
normal_df = make_df_with_genes(gene_list,cancer_list,type_dict,tissue_type = "normal")

In [ ]:
plt = sns.stripplot(x="Correlation",y="Gene",hue="Cancer Type",data=normal_df,jitter=0)
plt.legend_.remove()
plt.set(xlim=(-.5,1))
plt

In [ ]:
#Make sure colors are the same, make axes equal as well.
#Use parameter quiet when using the CPTAC package
#Add introductory readings to intro in outline
#Be somewhat familiar with genes and where they are talked about